In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data and display first 5 rows
z_data = pd.read_csv('esem_act_deact_zscore_groups.csv',sep=';',decimal='.')
z_data.drop(list(z_data.filter(regex = '_act')),axis = 1, inplace = True)
z_data.drop(list(z_data.filter(regex = 'aggr')),axis = 1, inplace = True)
data = z_data

#we need the list with the labels to test the predictions; everytime the label or proband changes, we add it to the list
# grouped = z_data.groupby(['proband', 'trial','task'])
# groupedAgg = grouped.aggregate(np.mean)
labelsComputingAccuracy = z_data.groupby(['proband', 'trial','task']).aggregate(np.mean).index.get_level_values(level='task')
labels = z_data.task

#=== remove everything that is not a feature ===================================================
z_data = z_data.drop('proband', axis = 1)
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('task', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = z_data.columns

# Convert to numpy array
features = np.array(z_data)
labels = np.array(labels)

numLabels = 3 # num of different labels (comprehension and rest)
sizeTrainSet = 13 # num of participants used for training
sizeTestSet = 4 # num participants used for testing
 
# get the number of trials per participant
numScans = len(data.scan.unique())
numParticipants = sizeTrainSet + sizeTestSet


#get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
training_features = features[0:numScans*sizeTrainSet]
training_features = np.array(training_features)
testing_features = features[numScans*sizeTrainSet:numScans*numParticipants]
testing_features = np.array(testing_features)

training_target = labels[0:numScans*sizeTrainSet]
training_target = np.array(training_target)
testing_target = labels[numScans*sizeTrainSet:numScans*numParticipants]
testing_target = np.array(testing_target)



In [11]:
print(training_features.shape)
print(testing_features.shape)
print(training_target.shape)
print(testing_target.shape)
print(len(labelsComputingAccuracy))
print(numScans)
type(training_features)
z_data.head(5)

(8710, 251)
(2680, 251)
(8710,)
(2680,)
1020
670


,113-159-85_deact,113-162-85_deact,113-156-88_deact,113-159-88_deact,113-162-88_deact,113-156-91_deact,113-159-91_deact,113-162-91_deact,116-156-85_deact,116-159-85_deact,...,137-186-109_deact,140-180-103_deact,140-183-103_deact,140-183-106_deact,proband,scan,trial,task,snippet,response
0,-0.595409,-0.415868,0.128070,0.375606,0.796242,1.790735,1.896647,1.547686,-0.422856,-0.006433,...,1.795029,0.146092,0.628055,0.868627,bd96,0,0,0,arrayAverage,168147.0
1,-0.330659,0.017706,0.370144,0.313801,0.455320,1.561344,1.295762,0.827522,-0.366268,0.063675,...,1.385217,-0.416112,0.042862,0.044537,bd96,1,0,0,arrayAverage,168147.0
2,0.342433,1.018949,0.625187,0.437411,0.157553,0.804354,0.328925,0.055610,-0.060026,0.340766,...,0.857380,-0.530432,-0.058034,-0.589379,bd96,2,0,0,arrayAverage,168147.0
3,0.764238,1.045768,0.335562,0.702289,0.563207,-0.237081,0.053332,0.512720,0.629017,0.567780,...,0.916393,-0.228189,-0.002541,-0.394133,bd96,3,0,0,arrayAverage,168147.0
4,0.373844,0.647953,-0.887778,-1.182759,-1.111193,-1.347333,-1.514390,-1.613272,-0.622579,-0.684137,...,-0.729411,-0.881222,-1.041764,-1.274007,bd96,4,0,0,arrayAverage,168147.0


In [3]:
from tpot import TPOTClassifier
fileNameTPot = 'Deact/tpot_mnist_pipeline_NotAggregatedParticipantSplit_Act'

tpot = TPOTClassifier(generations=5, population_size=1, verbosity=2,n_jobs = 20)
tpot.fit(training_features, training_target)
print(tpot.score(testing_features, testing_target))
tpot.export(fileNameTPot + '.py')

C:\Users\author\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\author\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnin

Generation 1 - Current best internal CV score: 0.4505166475315729


Generation 2 - Current best internal CV score: 0.4505166475315729


Generation 3 - Current best internal CV score: 0.4505166475315729


Generation 4 - Current best internal CV score: 0.4505166475315729


Generation 5 - Current best internal CV score: 0.4505166475315729



Best pipeline: GaussianNB(DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=8, min_samples_leaf=20, min_samples_split=10))
0.4126865671641791


True

In [10]:
print(training_features.shape)
print(testing_features.shape)
print(training_target.shape)
print(testing_target.shape)
print(len(labelsComputingAccuracy))
print(len(training_features))
z_data.head(5)

(8710, 251)
(2680, 251)
(8710,)
(2680,)
1020
8710


,113-159-85_deact,113-162-85_deact,113-156-88_deact,113-159-88_deact,113-162-88_deact,113-156-91_deact,113-159-91_deact,113-162-91_deact,116-156-85_deact,116-159-85_deact,...,137-186-109_deact,140-180-103_deact,140-183-103_deact,140-183-106_deact,proband,scan,trial,task,snippet,response
0,-0.595409,-0.415868,0.128070,0.375606,0.796242,1.790735,1.896647,1.547686,-0.422856,-0.006433,...,1.795029,0.146092,0.628055,0.868627,bd96,0,0,0,arrayAverage,168147.0
1,-0.330659,0.017706,0.370144,0.313801,0.455320,1.561344,1.295762,0.827522,-0.366268,0.063675,...,1.385217,-0.416112,0.042862,0.044537,bd96,1,0,0,arrayAverage,168147.0
2,0.342433,1.018949,0.625187,0.437411,0.157553,0.804354,0.328925,0.055610,-0.060026,0.340766,...,0.857380,-0.530432,-0.058034,-0.589379,bd96,2,0,0,arrayAverage,168147.0
3,0.764238,1.045768,0.335562,0.702289,0.563207,-0.237081,0.053332,0.512720,0.629017,0.567780,...,0.916393,-0.228189,-0.002541,-0.394133,bd96,3,0,0,arrayAverage,168147.0
4,0.373844,0.647953,-0.887778,-1.182759,-1.111193,-1.347333,-1.514390,-1.613272,-0.622579,-0.684137,...,-0.729411,-0.881222,-1.041764,-1.274007,bd96,4,0,0,arrayAverage,168147.0


In [5]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content] 

#      or 'exported_pipeline = ' not in line
cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [6]:
%run -i $fileForLearning

In [7]:
z_data = data
condition = z_data.at[sizeTrainSet*numScans, 'task']
participant = z_data.at[sizeTrainSet*numScans, 'proband']
majorityVote = []
countComprehension = 0
countRest = 0
countD2 = 0
    
for i,j in zip(range(sizeTrainSet*numScans,len(z_data)),range(len(results))):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
        if (countComprehension > countRest) and (countComprehension > countD2):
            majorityVote.append('C')
        elif countD2 > countRest and countD2 > countComprehension:
                majorityVote.append('D2')
        elif countRest > countComprehension and countRest > countD2:
            majorityVote.append('R')
        elif countComprehension == countRest:
            majorityVote.append('CR')
        elif countComprehension == countD2:
            majorityVote.append('CD2')
        else:
            majorityVote.append('D2R')
        countComprehension = 0
        countRest = 0
        countD2 = 0

    if (results[j] == 0):
        countComprehension = countComprehension + 1
    if (results[j] == 1):
        countRest = countRest + 1
    if (results[j] == 2):
        countD2 = countD2 + 1

if (results[j] == 0):
    countComprehension = countComprehension + 1
if (results[j] == 1):
    countRest = countRest + 1
if (results[j] == 2):
    countD2 = countD2 + 1

if (countComprehension > countRest) and (countComprehension > countD2):
    majorityVote.append('C')
elif countD2 > countRest and countD2 > countComprehension:
        majorityVote.append('D2')
elif countRest > countComprehension and countRest > countD2:
    majorityVote.append('R')
elif countComprehension == countRest:
    majorityVote.append('CR')
elif countComprehension == countD2:
    majorityVote.append('CD2')
else:
    majorityVote.append('D2R')


In [8]:
labelsComputingAccuracy = pd.Series(labelsComputingAccuracy)
labelsComputingAccuracy = labelsComputingAccuracy.replace({0: 'C', 1: 'R', 2: 'D2'})

num_matches = 0;
for a, b in zip(majorityVote, labelsComputingAccuracy):
    if b in a:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',len(labelsComputingAccuracy),')')

accuracy = num_matches/len(labelsComputingAccuracy)*100
print('Accuary: ',accuracy)


Number of matches: 106 (of 1020 )
Accuary:  10.392156862745098


In [8]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'NoAgg_Deact' + str(accuracy) + '\n')
